## Код Хэмминга (n,k)

- $n$ - число бит в кодовой последовательности
- $k$ - число информационных бит
- $r = n - k$ - число контрольных бит


In [1]:
import numpy as np
from sympy import *
from functools import reduce
from IPython.display import Math

### Пример для (7,4)

In [2]:
n,k = (7,4)
r = n - k

Контрольные биты четности $p_{1..r}$ вычисляются из битов данных $d_{1..k}$ следующим образом:

> $p_1 = d_1 \oplus d_2 \oplus d_4$  
> $p_2 = d_1 \oplus d_3 \oplus d_4$  
> $p_3 = d_2 \oplus d_3 \oplus d_4$  

Вектор кодовой последовательности $\langle d_1, d_2, d_3, d_4, p_1, p_2, p_3 \rangle$ можно получить умножением вектора исходных данных $\langle d_1, d_2, d_3, d_4 \rangle$ на матрицу-генератор $G$, где $P$ - матрица перестановки, соответствующая вышеупомянутым комбинациям информационных битов для вычисления битов четности:

> $G = \begin{bmatrix}I_{k} ~\vdots~ P\end{bmatrix}$

Для проверки наличия ошибок в кодовой последовательности вычисляется вектор синдромов $\vec{S} = \langle S_1, S_2, S_3 \rangle$, такой что:
 
> $S_1 = p_1 \oplus d_1 \oplus d_2 \oplus d_4$  
> $S_2 = p_2 \oplus d_1 \oplus d_3 \oplus d_4$  
> $S_3 = p_3 \oplus d_2 \oplus d_3 \oplus d_4$  

Если вектор синдромов равен нулевому вектору $\langle 0, 0, 0 \rangle$, то кодовая последовательность не содержит ошибок. Вектор синдромов можно получить умножением вектора кодовой последовательности на матрицу контроля четности:

> $H = \begin{bmatrix}P \\ \cdots \\ I_{r}\end{bmatrix}$
 

---
#### Матрица-генератор:

In [3]:
Ik = np.eye(k, dtype='B')
P = np.array([[1,1,0],[1,0,1],[0,1,1],[1,1,1]], dtype='B')

G = np.hstack((Ik, P))

display(Math('G = ' + latex(Matrix(G))))

<IPython.core.display.Math object>

#### Матрица контроля чётности:

In [4]:
Ir = np.eye(r, dtype='B')
H = np.vstack((P, Ir))

display(Math('H = ' + latex(Matrix(H))))

<IPython.core.display.Math object>

#### Произвольный вектор с информационными битами: $\vec{D}$

In [5]:
D = np.random.randint(0, 2, (1, k), dtype='B')

display(Math('\\vec{D} = ' + latex(Matrix(D))))

<IPython.core.display.Math object>

#### Вычисление кодовой последовательности: $\vec{C} = \vec{D} \times G$

In [6]:
C = np.dot(D, G) % 2

display(Math('\\vec{C} = ' + latex(Matrix(C))))

<IPython.core.display.Math object>

#### Проверка кодовой последовательности, вычисление вектора синдромов: $\vec{S} = \vec{C} \times H$

In [7]:
S = np.dot(C, H) % 2

display(Math('\\vec{S} = ' + latex(Matrix(S))))

<IPython.core.display.Math object>

---
#### Исправление ошибок в кодовой последовательности

Для обнаружения и исправления единичной ошибки в кодовой последовательности удобно распологать в ней биты в таком порядке, чтобы контрольные биты $p_{1..r}$ находились на позициях кратных степени двойки: $2^{x}, \{ x \in \mathbb{N}, 0 \leq x \lt r \}$.
В этом случае полученный при декодировании вектор синдромов $\langle S_1, S_2, S_3 \rangle$ будет являться двоичным представлением индекса ошибочного символа в кодовой последовательности.

Переставим столбцы в матрице-генераторе и строки в матрице контроля четности таким образом, чтобы результирующий вектор кодовой последовательности содержал биты в упомянутом порядке:

> $\langle d_1, d_2, d_3, d_4, p_1, p_2, p_3 \rangle \mapsto \langle p_1, p_2, d_1, p_3, d_2, d_3, d_4 \rangle$

In [8]:
Gp = G[:,(4,5,0,6,1,2,3)]
Hp = H[(4,5,0,6,1,2,3),:]

display(Math('Gp = ' + latex(Matrix(Gp)) +'; ~~~ Hp = ' + latex(Matrix(Hp))))

<IPython.core.display.Math object>

Вычислим кодовую последовательность и синдром используя обновленные матрицы:

In [9]:
Cp = np.dot(D, Gp) % 2
Sp = np.dot(Cp, Hp) % 2
display(Math('\\vec{Cp} = ' + latex(Matrix(Cp))))
display(Math('\\vec{Sp} = ' + latex(Matrix(Sp))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Тест: Имитация единичной ошибки в кодовой последовательности и вычисление индекса ошибочного символа:

In [10]:
for i in range(7):
    Cp[0,i] ^= 1
    Sp = np.dot(Cp, Hp) % 2
    Cp[0,i] ^= 1
    err = np.packbits(Sp, bitorder='little')
    display(Math("\\vec{Sp} = " + latex(Matrix(Sp)) + " \\implies \\text{ошибка в символе}~\\vec{Cp_{%d}}" % err))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>